# Accuracy

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tf.enable_eager_execution()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

AUTOTUNE = tf.data.experimental.AUTOTUNE

## Load index csv

In [2]:
dataset_index = pd.read_csv('./dataset/tiles-data/data.csv')
print(dataset_index.shape)
print(dataset_index.head())
print('\n')

dataset_size = dataset_index.shape[0]
num_classes = np.amax(dataset_index['label'])

# append path prefix
dataset_index['image-path'] = './dataset/tiles-resized/' + dataset_index['image-name']

print('append path prefix')
print(dataset_index.shape)
print(dataset_index.head())
print('\n')

# subtract label id by 1
dataset_index['label'] -= 1

test_index = dataset_index
test_size = test_index.shape[0]

print('subtract label id by 1')
print(test_index.shape)
print(test_index.head())
print('\n')

(629, 3)
  image-name  label    label-name
0      1.jpg     38  bonus-winter
1      2.jpg     26  characters-8
2      3.jpg      9        dots-9
3      4.jpg     36  bonus-summer
4      5.jpg     28   honors-east


append path prefix
(629, 4)
  image-name  label    label-name                     image-path
0      1.jpg     38  bonus-winter  ./dataset/tiles-resized/1.jpg
1      2.jpg     26  characters-8  ./dataset/tiles-resized/2.jpg
2      3.jpg      9        dots-9  ./dataset/tiles-resized/3.jpg
3      4.jpg     36  bonus-summer  ./dataset/tiles-resized/4.jpg
4      5.jpg     28   honors-east  ./dataset/tiles-resized/5.jpg


subtract label id by 1
(629, 4)
  image-name  label    label-name                     image-path
0      1.jpg     37  bonus-winter  ./dataset/tiles-resized/1.jpg
1      2.jpg     25  characters-8  ./dataset/tiles-resized/2.jpg
2      3.jpg      8        dots-9  ./dataset/tiles-resized/3.jpg
3      4.jpg     35  bonus-summer  ./dataset/tiles-resized/4.jpg
4      5

## Get dataset, preprocessing

In [3]:
def show_image_sample(ds):
    for n, (image, label) in enumerate(ds.take(4)):
        plt.subplot(2,2,n+1)
        plt.imshow(image)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(np.argmax(label))
    plt.show()

def read_image(path):
    image = tf.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [320, 320])
    image /= 255.0 # normalize to [0,1]
    return image

def rotate_image(image, label):
    image = tf.image.rot90(image)
    return image, label

def get_dataset(dataset_index):
    path_ds = tf.data.Dataset.from_tensor_slices(dataset_index['image-path'])
    image_ds = path_ds.map(read_image, num_parallel_calls=AUTOTUNE)
    
    label_one_hot = tf.one_hot(dataset_index['label'], num_classes)
    label_ds = tf.data.Dataset.from_tensor_slices(label_one_hot)
    
    paired_ds = tf.data.Dataset.zip((image_ds, label_ds))
    
    paired_ds_90 = paired_ds.map(rotate_image, num_parallel_calls=AUTOTUNE)
    paired_ds_180 = paired_ds_90.map(rotate_image, num_parallel_calls=AUTOTUNE)
    paired_ds_270 = paired_ds_180.map(rotate_image, num_parallel_calls=AUTOTUNE)
    paired_ds = paired_ds.concatenate(paired_ds_90).concatenate(paired_ds_180).concatenate(paired_ds_270)

    return paired_ds
    
test_ds = get_dataset(test_index)
test_size *= 4
print('test size', test_size)

Instructions for updating:
Colocations handled automatically by placer.
test size 2516


## Load classifier

In [4]:
encoder = keras.models.load_model('./models/encoder_final.h5')
fc = keras.models.load_model('./models/fc_final.h5')

# Prevent training
for layer in encoder.layers:
    layer.trainable = False
    
for layer in fc.layers:
    layer.trainable = False

classifier_input = keras.Input(shape=(320,320,3))
classifier_output = fc(encoder(classifier_input))
classifier = keras.Model(inputs=classifier_input, outputs=classifier_output)

print(classifier.summary())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 3)       0         
_________________________________________________________________
encoder (Sequential)         (None, 16, 16, 16)        18656     
_________________________________________________________________
fc (Sequential)              (None, 42)                809066    
Total params: 827,722
Trainable params: 0
Non-trainable params: 827,722
_________________________________________________________________
None


## Predict

In [23]:
batch_size = 32

def add_dimension(image, label):
    image = image[np.newaxis,:]
    return image, label

test_ds_classes = {}
predicts = {}

for i in range(num_classes):
    test_ds_classes[i] = test_ds.filter(lambda x, y: y == i)\
                                .map(add_dimension, num_parallel_calls=AUTOTUNE)\
                                .cache()\
                                .apply(tf.data.experimental.shuffle_and_repeat(buffer_size=test_size))\
                                .prefetch(buffer_size=AUTOTUNE)

# for cls, ds in test_ds_classes.items():
#     predicts[cls] = classifier.predict(ds, 1)

for cls in range(5):
    predicts[cls] = classifier.predict(test_ds_classes[cls], steps=1)

print(predicts)

{0: [], 1: [], 2: [], 3: [], 4: []}
